In [1]:
using DataFrames
using Statistics
using Distributions
using DataStructures
using Parameters

## W niniejszym notebooku zostanie dokonana symulacja działania lodowiska.

Początkowo poczynimy następujace założenia:
* jest jedna kasa
* kasa obsługuje klientów pojedynczo
* czas obsługi klienta jest zmienną losową o rozkładzie wykładniczym, którego parametr będzie zmienną wejściową
* czas pobytu gościa na lodowisku również będzie zmienną losową o powyższym rozkładzie o parametrze wprowadzanym przez zmienną wejściową
* Gdy kasa jest zajęta, klient czeka w kolejce
* Klient chce wejść na lodowisko za wszelką cenę - nie opuści kolejki niezależnie od jej długości
* Klient z pewnym prawdopodobieństwem wypożyczy łyżwy - parametr wejściowy


### Pierwszy etap:
* jedna kasa
* kolejka do kasy
* klient nie opuszcza kolejki niezależnie od jej długości
* kolejka obsługuje jedną osobę, obsługa klienta ma czas zgodny z rozkładem wykładniczym


In [ ]:
@with_kw mutable struct Lodowisko
    tick::Int64 = 0
    people_on_rink_T::Int64 = 0
    p_o_r_total::DataFrame = DataFrame(From=Float64[], To = Float64[], number = Int[])
    
    
    isKASAidle::Bool=true
    served_clientsK::Int64 = 0
    when_KASA_idle_when_not::DataFrame = DataFrame(From = Float64[], To = Float64[], IDLE = Bool[])
    queue::PriorityQueue{Function, Tuple{Float64, Int}} = PriorityQueue{Function, Tuple{Float64, Int}}()    
    
    queue_to_kasa::Queue{Float64} = Queue{Float64}()
    
    isRENTidle::Bool=true
    served_clients_rent::Int64 = 0
    money_earned::Int64 = 0
    prob_of_rent::Float64 = 0.7
    price::Int64 = 10
    
    waittime::DataFrame = DataFrame(From = Float64[], To = Float64[])
end

# @with_kw mutable struct Kasa
#     isKASAidle::Bool=true
#     served_clientsK::Int64 = 0
#     when_KASA_idle_when_not::DataFrame = DataFrame(From = Int[], To = Int[], IDLE = Bool[])
#     queue_kasa::PriorityQueue{Function, Int64} = PriorityQueue{Function, Int64}()
# end

# @with_kw mutable struct Rent
#     isRENTidle::Bool=true
#     served_clients_rent::Int64 = 0
#     money_earned::Int64 = 0
# end

In [3]:
function client_arrived!(sim, T)
    if sim.isKASAidle == true
        sim.isKASAidle = false
        t_of_service = rand(Exponential(75))
        push!(sim.when_KASA_idle_when_not, (T, T + t_of_service, true))
        enqueue!(sim.queue, kasa_served!, (T + t_of_service, 1))
    else
        enqueue!(sim.queue_to_kasa, T)
        
    end
        
end



function kasa_check!(sim, T)
    s = length(sim.queue_to_kasa)
    if s != 0
        t_serv = dequeue!(sim.queue_to_kasa)
        push!(sim.waittime, (t_serv, T))
        sim.isKASAidle = false
        t_of_service = rand(Exponential(75))
        push!(sim.when_KASA_idle_when_not, (T, T+t_of_service, true))
        enqueue!(sim.queue, kasa_served!, (T + t_of_service, 1))
    end
end



function kasa_served!(sim, T)
    sim.isKASAidle = true
    sim.served_clientsK += 1
    enqueue!(sim.queue, kasa_check!, (T, 7))
end

kasa_served! (generic function with 1 method)

In [4]:
function step!(sim)
    if sim.tick == 0
        t_ar = rand(Exponential(75))
        enqueue!(sim.queue, client_arrived!, (t_ar, 1))
    end
    sim.tick+=1
    
    func, time_tuple = dequeue_pair!(sim.queue)
    time = time_tuple[1]
    func(sim, time)
    
    if String(Symbol(func)) == "client_arrived!"
        t_ar = rand(Exponential(75))
        enqueue!(sim.queue, client_arrived!, (time + t_ar, 1))
        
    end
    return time
end

step! (generic function with 1 method)

In [5]:
model = Lodowisko()
horizon = 3600*8

while true
    t = step!(model)
    if t >= horizon
        break
    end
end


In [6]:
model.waittime

,From,To
,Float64,Float64
1,288.901,328.822
2,476.824,660.833
3,514.942,770.291
4,531.53,780.471
5,556.111,853.64
6,688.54,866.648
7,710.721,1046.5
8,856.362,1075.83
9,865.693,1120.31


### II etap:

* dodajemy lodowisko

In [3]:
@with_kw mutable struct Lodowisko2
    tick::Int64 = 0
    people_on_rink_T::Int64 = 0
    p_o_r_total::DataFrame = DataFrame(From=Float64[], To = Float64[], number = Int[])
    last_change_p_o_r::Float64 = 0
    
    isKASAidle::Bool=true
    served_clientsK::Int64 = 0
    when_KASA_idle_when_not::DataFrame = DataFrame(From = Float64[], To = Float64[], IDLE = Bool[])
    queue::PriorityQueue{Function, Tuple{Float64, Int}} = PriorityQueue{Function, Tuple{Float64, Int}}()    
    
    queue_to_kasa::Queue{Float64} = Queue{Float64}()
    
    isRENTidle::Bool=true
    served_clients_rent::Int64 = 0
    money_earned::Int64 = 0
    prob_of_rent::Float64 = 0.7
    price::Int64 = 10
    
    waittime::DataFrame = DataFrame(From = Float64[], To = Float64[])
    
    times_of_spends::DataFrame = DataFrame(ID = Int[], From = Float64[], To = Float64[])
    
end

Lodowisko2

In [11]:
function client_arrived2!(sim, T)
    if sim.isKASAidle == true
        sim.isKASAidle = false
        t_of_service = rand(Exponential(75))
        push!(sim.when_KASA_idle_when_not, (T, T + t_of_service, true))
        enqueue!(sim.queue, kasa_served2!, (T + t_of_service, 2))
    else
        enqueue!(sim.queue_to_kasa, T)
        
    end
        
end

function kasa_served2!(sim, T)
    sim.isKASAidle = true
    sim.served_clientsK += 1
    enqueue!(sim.queue, kasa_check2!, (T, 3))
    enqueue!(sim.queue, client_enters_rink!, (T, 4))
end

function kasa_check2!(sim, T)
    s = length(sim.queue_to_kasa)
    if s != 0
        t_serv = dequeue!(sim.queue_to_kasa)
        push!(sim.waittime, (t_serv, T))
        sim.isKASAidle = false
        t_of_service = rand(Exponential(75))
        push!(sim.when_KASA_idle_when_not, (T, T+t_of_service, true))
        enqueue!(sim.queue, kasa_served2!, (T + t_of_service, 2))
    end
end


function client_enters_rink!(sim, T)

    time_last_change = sim.last_change_p_o_r
     
    push!(sim.p_o_r_total, (time_last_change, T, sim.people_on_rink_T))
    
    sim.last_change_p_o_r = T
    sim.people_on_rink_T+=1
    time_spent = rand(Exponential(3600))

    push!(sim.times_of_spends, (sim.people_on_rink_T, T, T+time_spent))
    
    #enqueue!(sim.queue, client_leaves!, (T + time_spent, 5))
end

# function client_leaves!(sim, T)
    
#     time_last_change = sim.last_change_p_o_r
     
#     push!(sim.p_o_r_total, (time_last_change, T, sim.people_on_rink_T))
    
#     sim.last_change_p_o_r = T
#     sim.people_on_rink_T-=1
    
# end

client_enters_rink! (generic function with 1 method)

In [12]:
function step2!(sim::Lodowisko2)
    if sim.tick == 0
        t_ar = rand(Exponential(180))
        enqueue!(sim.queue, client_arrived2!, (t_ar, 1))
    end
    sim.tick+=1
    
    #println(sim.queue)
    func, time_tuple = dequeue_pair!(sim.queue)
    time = time_tuple[1]
    func(sim, time)
    
    if String(Symbol(func)) == "client_arrived2!"
        t_ar = rand(Exponential(180))
        enqueue!(sim.queue, client_arrived2!, (time + t_ar, 1))
        
    end
    return time
end

step2! (generic function with 1 method)

In [46]:
modelx = Lodowisko2()
horizon = 3600*8

while true
    t = step2!(modelx)
    if t >= horizon
        break
    end
end


In [47]:
modelx.times_of_spends

,ID,From,To
,Int64,Float64,Float64
1,1,306.723,403.192
2,2,513.883,8348.08
3,3,813.092,2597.77
4,4,924.445,11575.3
5,5,1041.2,1468.38
6,6,1243.73,2182.42
7,7,1535.1,7894.17
8,8,1635.87,5013.44
9,9,1937.79,2147.6


In [48]:
entrances = DataFrame(timestamp = modelx.times_of_spends.From, aux = ones(length(modelx.times_of_spends.ID)))
exits = DataFrame(timestamp = modelx.times_of_spends.To, aux = -1*ones(length(modelx.times_of_spends.ID)))
all_events = DataFrame()
append!(all_events, entrances)
append!(all_events, exits)
sort!(all_events)

,timestamp,aux
,Float64,Float64
1,306.723,1.0
2,403.192,-1.0
3,513.883,1.0
4,813.092,1.0
5,924.445,1.0
6,1041.2,1.0
7,1243.73,1.0
8,1468.38,-1.0
9,1535.1,1.0


In [54]:
all_events.g_num = cumsum(all_events.aux);

In [50]:
all_events

,timestamp,aux,g_num
,Float64,Float64,Float64
1,306.723,1.0,1.0
2,403.192,-1.0,0.0
3,513.883,1.0,1.0
4,813.092,1.0,2.0
5,924.445,1.0,3.0
6,1041.2,1.0,4.0
7,1243.73,1.0,5.0
8,1468.38,-1.0,4.0
9,1535.1,1.0,5.0


In [51]:
mean(all_events.g_num)

19.766272189349113

## W kolejnych etapach należałoby dodać parametry wejściowe dla parametrów rozkładów czasu obsługi, czasu przyjścia kolejnego klienta oraz czasu pobytu na lodowisku. Ponadto symulacja na razie nie pozwala na skrócenie czasu pobytu na lodowisku w przypadku skończenia się horyzontu czasowego. Lodowisko działa do ostatniego klienta

In [52]:
last(all_events)

,timestamp,aux,g_num
,Float64,Float64,Float64
338,44119.5,-1.0,0.0


In [53]:
43898.2/3600

12.193944444444444